
# 🔧Samsung DocuMate- Your AI companion for Samsung docs.



## 📌 Objective
Transform traditional, static product manuals into an interactive, intelligent support experience. 

With the power of **RAG**, users can simply type a question and instantly get precise answers — pulled straight from the official manual, along with visual page previews.

---

## 🚀 Why This Is Helpful

- ✨ **Smart Support**: Empowers users to solve issues on their own through AI-guided responses.
- 
- 🔍 **Fast & Accurate**: Uses semantic retrieval to fetch only the most relevant content.
- 
- 🖼️ **Visual Assistance**: Provides page previews of relavant PDF Pages for better understanding.
- 
- 📦 **Model-Agnostic**: Works with any Samsung device — just enter the model number!

---

💡 Whether you're troubleshooting, exploring features, or looking for quick how-tos, this assistant brings manuals to life — turning tech confusion into clarity.


Links
Blog Link: https://medium.com/@21f1002451/samsung-documate-your-ai-companion-for-samsung-docs-a150b16a1ef1

Youtube Link: https://www.youtube.com/watch?v=kCJkCtCbv4Y


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import json# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install selenium pymupdf langchain-google-genai gradio faiss-cpu
!pip install langchain-community  # for PyPDFLoader & FAISS vectorstore


In [ ]:
!pip install beautifulsoup4 lxml



In [ ]:
import os
import re
import requests
import fitz  # PyMuPDF
import gradio as gr
from typing import List, Tuple, Dict,Optional

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from pydantic import PrivateAttr


In [ ]:
BASE_URL = "https://www.samsung.com/in/support/user-manuals-and-guide/"
DOWNLOAD_DIR = "downloads"
DIAGRAM_DIR = "diagrams"
PREVIEW_DIR = "previews"
for directory in (DOWNLOAD_DIR, DIAGRAM_DIR, PREVIEW_DIR):
    os.makedirs(directory, exist_ok=True)

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret = user_secrets.get_secret("API_KEY")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/kaggle/input/silver-fiber/silver-fiber-401514-75aa42fb5a08.json"
os.environ["GENAI_API_KEY"] = secret

### `GenAILLM(api_key: str, model_name: str = None, temperature: float = None)`

**Purpose:**  
Serves as a LangChain-compatible wrapper for integrating Google Generative AI (Gemini models) into LLM pipelines, enabling seamless prompt execution and response handling.

---

**Why is this needed?**

- **LangChain Integration:**  
  Allows Google GenAI models to be used as drop-in replacements within LangChain’s toolchains.

- **Custom Model Control:**  
  Supports dynamic selection of model type and temperature tuning for flexibility across use cases.

- **Cleaner Abstraction:**  
  Provides a structured and reusable interface to send prompts and retrieve responses using Google's GenAI client.

---

**How does this work?**

1. **Initialization:**
   - Takes in a Google API key and optional configuration parameters.
   - Initializes the GenAI client via `genai.Client`.

2. **Prompt Execution:**
   - Overrides the `_call()` method to submit the prompt to the GenAI model and return the plain text response.

3. **Metadata Exposure:**
   - Implements `_identifying_params` to return the model’s configuration for reproducibility.
   - Specifies `_llm_type` as `"genai"` for internal tracking within LangChain.

This class enables smooth and structured use of Gemini models in any pipeline requiring LLM-backed responses.



In [ ]:
# Custom LangChain-compatible wrapper for Google's Generative AI (Gemini models)
class GenAILLM(LLM):
    model_name: str = "gemini-2.0-flash"   # Default model to use
    temperature: float = 0.0               # Default temperature for deterministic output
    _client: any = PrivateAttr()           # Internal GenAI client (not exposed publicly)

    def __init__(self, api_key: str, model_name: str = None, temperature: float = None):
        from google import genai
        super().__init__()  # Initialize base LLM class
        self._client = genai.Client(api_key=api_key)  # Instantiate GenAI client using the API key

        # Optionally override default model and temperature
        if model_name:
            self.model_name = model_name
        if temperature is not None:
            self.temperature = temperature

    def _call(self, prompt: str, stop=None) -> str:
        """
        Core function to send a prompt to the LLM and return the generated response text.
        """
        response = self._client.models.generate_content(
            model=self.model_name,
            contents=prompt,
        )
        return response.text

    @property
    def _identifying_params(self) -> Dict:
        """
        Provides identifying parameters for LangChain to distinguish LLM configurations.
        """
        return {"model_name": self.model_name, "temperature": self.temperature}

    @property
    def _llm_type(self) -> str:
        """
        Returns the type label used internally by LangChain.
        """
        return "genai"

# Custom exception to raise when no English manual is available for a given model
class NoEnglishManualError(Exception):
    """Raised when no English PDF user manual is available for a model."""
    pass


### `setup_parser()` Function

**Purpose:**  
Creates a structured parser and customized prompt template to ensure consistent and accurate responses from an LLM when answering technical questions based solely on provided manual context.

---

**Why is it needed?**

- **Structured Responses:**  
  Ensures the LLM’s answers follow a predictable, JSON-like format, making it easier to integrate and use responses programmatically.

- **Clear Instructions for LLM:**  
  Provides explicit guidelines, emphasizing detailed explanations and avoiding the generation of unsupported content (like diagrams not in the context).

- **Improved Reliability:**  
  By clearly defining response expectations, the function reduces model hallucinations and enhances the reliability of the generated answers, especially useful in precise applications like technical manual queries.



In [ ]:
def setup_parser() -> Tuple[StructuredOutputParser, PromptTemplate]:
    """
    Sets up a structured output parser and a customized prompt template for use in a
    Retrieval-Augmented Generation (RAG) question-answering pipeline.

    Returns:
        Tuple[StructuredOutputParser, PromptTemplate]: The parser for formatting outputs
        and the prompt template that guides the LLM's response behavior.
    """

    # Define the expected schema for the LLM's structured output
    schemas = [ResponseSchema(name="answer", description="The answer to the user's question.")]

    # Create a structured parser based on the schema
    parser = StructuredOutputParser.from_response_schemas(schemas)

    # Generate format instructions that tell the LLM how to structure its response
    format_instructions = parser.get_format_instructions()

    # Define a custom prompt template with embedded format instructions and contextual placeholders
    prompt_template = PromptTemplate(
        template="""
You are a helpful and precise assistant specialized in answering questions based on technical manuals.

Use the following EXACT format for your answer and Do NOT include source:
{format_instructions}

Your task is to:
- Provide a clear and **in-depth answer** to the user’s question using only the given manual context.
- If the answer involves steps or instructions, **explain them in detail** (step-by-step if applicable).
- If a diagram is *not available* in the context, do NOT invent one.
Manual Context:
{context}

Question: {question}
""",
        input_variables=["context", "question"],               # Inputs the prompt expects
        partial_variables={"format_instructions": format_instructions}  # Injects format rules directly
    )

    # Return both the parser and the prompt for use in the RAG pipeline
    return parser, prompt_template

# Initialize the output parser and prompt once
output_parser, prompt = setup_parser()


### `_extract_json_array(text: str, key: str) → List[dict]`

**Purpose:**  
Efficiently extract and decode a JSON array associated with a given key from raw webpage content.

---

**Why is it needed?**  
When scraping web data, JSON content often appears embedded directly in the webpage as a plain string. This function ensures accurate extraction of just the JSON array you care about, converting it into a Python-friendly structure for easy data manipulation.

---

**How does it work?**

- **Pattern Matching:**  
  Uses regular expressions (`re`) to precisely locate the beginning of the JSON array (based on the provided key).

- **Balanced Bracket Parsing:**  
  Once it finds the array start, the function carefully tracks nested brackets (`[` and `]`) to find exactly where the JSON array ends, ensuring accurate extraction even for deeply nested JSON arrays.

- **JSON Decoding:**  
  Extracts the substring representing the JSON array and decodes it directly into a Python list of dictionaries using `json.loads()`.


In [ ]:
def _extract_json_array(text: str, key: str) -> List[dict]:
    """
    Extracts and returns a JSON array corresponding to a specific key from a raw text string.
    This is useful when JSON content is embedded inside HTML or JavaScript in a webpage.

    Args:
        text (str): The raw text (typically HTML or JS) containing embedded JSON.
        key (str): The key whose associated JSON array needs to be extracted.

    Returns:
        List[dict]: A list of dictionaries representing the parsed JSON array.

    Raises:
        RuntimeError: If the key is not found or if the array is not properly closed.
    """

    # Build regex pattern to find the start of the JSON array associated with the key
    pattern = f'"{key}"\\s*:\\s*\\['
    m = re.search(pattern, text)

    # If the key is not found in the text, raise an error
    if not m:
        raise RuntimeError(f"No '{key}' array found in page text.")

    # Start parsing from the beginning of the array (i.e., just before the first “[”)
    start = m.end() - 1
    depth = 0

    # Iterate over the characters starting from the array's opening bracket
    for i, ch in enumerate(text[start:], start=start):
        if ch == '[':
            depth += 1  # Increment depth for every opening bracket
        elif ch == ']':
            depth -= 1  # Decrement depth for every closing bracket

            # When depth returns to zero, the array is completely closed
            if depth == 0:
                # Extract the array substring and parse it into Python objects
                return json.loads(text[start:i+1])

    # If no matching closing bracket is found, raise an error
    raise RuntimeError(f"Could not find end of '{key}' array.")



### `get_manual_pdf_url(model_name: str)`

**Purpose:**  
Automatically retrieves the direct download link and filename of the English PDF user manual for a specific Samsung device model.

---

**Why is this needed?**

- **Automation of manual retrieval:**  
  Streamlines the process of finding and downloading specific device manuals, eliminating tedious manual searches.

- **Error Handling:**  
  Gracefully manages situations where the page doesn’t exist, doesn’t contain manual data, or doesn't have an English version available, thus ensuring robust behavior.

- **Efficient filtering:**  
  Precisely locates the English PDF manuals only, reducing clutter and enhancing the reliability of automated workflows.

---

**How does this work?**

1. **Request Page Content:**  
   - Constructs the support page URL using the provided `model_name`.
   - Sends an HTTP request with an appropriate user-agent header to fetch webpage content.

2. **Extract Manuals Data:**  
   - Parses the webpage text to extract JSON-formatted manual information using `_extract_json_array()`.

3. **Filter and Locate PDF Link:**  
   - Iterates through each manual entry, checking specifically for English language (`"EN"`).
   - Verifies that the manual description includes `"user manual"` and that the file is a `.pdf`.

4. **Returns Link & Filename:**  
   - If a valid English user manual PDF is found, returns its direct download URL along with the filename.
   - If no suitable manual is found or errors occur, returns `None`.

This function simplifies automated manual retrieval for further processing or immediate user access.


In [ ]:
def get_manual_pdf_url(model_name: str) -> Optional[Tuple[str, str]]:
    """
    Fetches the direct download URL and filename of the English PDF user manual
    for a given Samsung device model.

    Args:
        model_name (str): The Samsung model number (e.g., "SM-A166PLGBINS").

    Returns:
        Optional[Tuple[str, str]]: A tuple containing the PDF URL and filename,
        or None if no English user manual is found.
    """

    # Construct the product support page URL for the given model
    url = f"https://www.samsung.com/in/support/model/{model_name}/"
    headers = {"User-Agent": "Mozilla/5.0"}

    # Attempt to fetch the HTML content of the page
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        resp.raise_for_status()  # Raise an error if response is not 200 OK
    except requests.RequestException:
        return None  # Return None if the request fails

    # Try to extract the "manuals" JSON array embedded in the page
    try:
        manuals = _extract_json_array(resp.text, "manuals")
    except RuntimeError:
        return None  # Return None if no "manuals" key is found or extraction fails

    # Iterate through each manual entry to find a valid English user manual PDF
    for item in manuals:
        langs = item.get("languageList", [])
        
        # Check if English ("EN") is listed in the available languages
        if not any(lang.get("code") == "EN" for lang in langs):
            continue  # Skip manuals that aren't available in English

        # Extract the download URL and file name
        href  = item.get("downloadUrl")
        fname = item.get("fileName", "")
        desc  = item.get("englishDescription", "").lower()

        # Ensure it's a valid PDF user manual in English
        if href and fname.lower().endswith(".pdf") and "user manual" in desc:
            return href, fname  # Return the valid PDF URL and file name

    # If no suitable manual is found, return None
    return None


### `download_manual(href: str, out_path: str)`

**Purpose:**  
Downloads a PDF manual from a provided URL and saves it directly to the specified local file path.

---

**Why is this needed?**

- **Automated Retrieval:**  
  Enables automated, seamless downloading of user manuals directly from web resources without manual intervention.

- **Reliability Check:**  
  Verifies that the downloaded file is indeed a PDF, safeguarding against accidental downloads of incorrect or corrupted files.

- **Flexible Storage:**  
  Automatically manages directory creation, ensuring the target file location is always accessible, reducing filesystem-related errors.

---

**How does this work?**

1. **Fetch the PDF:**
   - Sends an HTTP GET request to the provided `href` URL.
   - Streams the file content efficiently to handle large PDF files.

2. **Validate Content-Type:**
   - Checks HTTP response headers to confirm the file type contains `"pdf"`.
   - Raises an error immediately if the file is not a PDF.

3. **Save to Local Storage:**
   - Ensures that the directory specified in `out_path` exists (creating it if necessary).
   - Writes the downloaded PDF data to the specified file path in manageable chunks (8KB each), optimizing memory usage.

4. **Return the Path:**
   - Returns the path where the manual has been successfully saved for further use.

This function is crucial for reliably handling manual downloads within an automated document retrieval and processing workflow.


In [ ]:
def download_manual(href: str, out_path: str) -> str:
    """
    Downloads a PDF manual from the provided URL and saves it to the specified local path.

    Args:
        href (str): Direct URL to the PDF manual.
        out_path (str): Path where the file should be saved locally.

    Returns:
        str: The full path to the downloaded PDF file.

    Raises:
        RuntimeError: If the downloaded file is not a PDF.
    """

    # Send a streaming GET request to the provided URL
    resp = requests.get(href, stream=True, timeout=10)
    resp.raise_for_status()  # Raise an exception for HTTP errors (e.g., 404, 500)

    # Check the content type to verify that it's a PDF
    content_type = resp.headers.get("Content-Type", "")
    if "pdf" not in content_type.lower():
        raise RuntimeError("Downloaded file is not a PDF.")

    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    # Write the PDF content to a file in chunks (8 KB each) to handle large files efficiently
    with open(out_path, "wb") as f:
        for chunk in resp.iter_content(1024 * 8):
            f.write(chunk)

    # Return the path to the saved PDF
    return out_path


### `extract_diagrams(pdf_path: str, output_dir: str = DIAGRAM_DIR)`

**Purpose:**  
Extracts all diagrams or visual content from a PDF file, saves them as image files (PNG), and returns a structured dictionary mapping page numbers to image paths.

---

**Why is this needed?**

- **Visual Insight Extraction:**  
  Many user manuals include diagrams that are crucial for understanding hardware layouts or instructions. This function helps extract and display those diagrams separately.

- **Supports RAG or Image-Based Display:**  
  Enables the downstream use of visuals in RAG systems, UI previews, or image-based retrieval for enhanced user interaction.

- **Fallback Handling:**  
  Ensures even if no images are embedded, the full page is rendered and saved—so that no visual context is missed.

---

**How does this work?**

1. **Prepare Output Directory:**
   - Ensures the directory to store extracted images exists.

2. **Open PDF and Iterate Pages:**
   - Loads the PDF using `PyMuPDF` (`fitz`) and loops through each page.

3. **Extract Images from Page:**
   - Uses `page.get_images(full=True)` to get all embedded images.
   - Converts images to RGB if needed and saves them as `pageX_imgY.png`.

4. **Fallback for Image-less Pages:**
   - If no embedded images are found, renders the **entire page** as a high-res image (`2x` zoom) and saves it as `pageX_full.png`.

5. **Organize Output:**
   - Stores image paths in a dictionary with page numbers as keys and lists of image file paths as values.

6. **Return Structure:**
   - Example output:
     ```python
     {
       "2": ["diagrams/page2_img1.png", "diagrams/page2_img2.png"],
       "5": ["diagrams/page5_full.png"]
     }
     ```

This function ensures that all visual content from the manual is extracted and available for downstream consumption in your GenAI workflows.


In [ ]:
def extract_diagrams(pdf_path: str, output_dir: str = DIAGRAM_DIR) -> Dict[str, List[str]]:
    """
    Extracts diagrams or visual content from each page of a PDF manual and saves them as PNG images.

    Args:
        pdf_path (str): Path to the PDF manual file.
        output_dir (str): Directory where extracted images will be saved.

    Returns:
        Dict[str, List[str]]: A dictionary mapping page numbers (as strings) to lists of image file paths.
    """

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Open the PDF using PyMuPDF (fitz)
    doc = fitz.open(pdf_path)
    
    # Dictionary to hold extracted image paths for each page
    diagrams: Dict[str, List[str]] = {}

    # Loop through each page in the document
    for idx, page in enumerate(doc):
        page_num = idx + 1

        # Attempt to extract embedded images from the page
        images = page.get_images(full=True)
        if images:
            for i, img in enumerate(images, start=1):
                xref = img[0]  # XREF ID of the image
                pix = fitz.Pixmap(doc, xref)  # Convert image to a Pixmap

                # Convert to RGB color space if necessary
                if pix.colorspace.name != 'DeviceRGB':
                    pix = fitz.Pixmap(fitz.csRGB, pix)

                # Save the image as a PNG file
                filename = f"page{page_num}_img{i}.png"
                path = os.path.join(output_dir, filename)
                pix.save(path)

                # Store image path in the dictionary under its page number
                diagrams.setdefault(str(page_num), []).append(path)

        else:
            # If no embedded images, render the full page as a high-resolution PNG
            pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # 2x zoom for clarity
            filename = f"page{page_num}_full.png"
            path = os.path.join(output_dir, filename)
            pix.save(path)

            # Store the rendered image path
            diagrams.setdefault(str(page_num), []).append(path)

    # Close the PDF file
    doc.close()

    # Return the dictionary of page-to-image mappings
    return diagrams


### `build_retriever(pdf_path: str)`

**Purpose:**  
Converts a PDF manual into a searchable vector database retriever, enabling semantic question-answering over its content using embeddings.

---

**Why is this needed?**

- **Foundational for RAG (Retrieval-Augmented Generation):**  
  This retriever powers the core of the QA system by allowing an LLM to access relevant document chunks instead of hallucinating.

- **Semantic Search Capabilities:**  
  Enables deep, context-aware retrieval from the manual—beyond keyword matching—by leveraging Google’s embedding model.

- **Ensures Data Quality:**  
  Verifies that the PDF is readable and not corrupted before processing, improving robustness.

---

**How does this work?**

1. **PDF Integrity Check:**
   - Uses `fitz` to confirm the PDF can be opened. Returns `None` if any issues are found.

2. **Load Document Content:**
   - Uses `LangChain`’s `PyPDFLoader` to extract text from the PDF pages.
   - If the PDF cannot be parsed (`PdfReadError`), it safely returns `None`.

3. **Split into Chunks:**
   - Uses `RecursiveCharacterTextSplitter` to break the document into overlapping chunks of ~1000 characters with 200-character overlap.
   - This helps preserve context across chunk boundaries during retrieval.

4. **Generate Embeddings:**
   - Applies Google’s `text-embedding-004` model to convert each chunk into a vector.

5. **Create Vector Store:**
   - Builds a FAISS index from the embedded chunks for fast similarity-based search.

6. **Return a Retriever:**
   - Returns a retriever object, which can be queried to fetch the most relevant chunks for any user question.

This function sets up the backbone for question answering based on the contents of any PDF manual.


In [ ]:
def build_retriever(pdf_path: str):
    """
    Builds a semantic retriever from a given PDF manual using LangChain components.
    This allows downstream systems to perform similarity-based retrieval on the document.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        Retriever object (or None): Returns a retriever instance if successful, or None if loading fails.
    """

    # Step 1: Verify PDF file integrity using PyMuPDF (fitz)
    try:
        import fitz
        fitz.open(pdf_path).close()  # Attempt to open and immediately close the file
    except Exception as e:
        return None  # Return None if the file is invalid or unreadable

    # Step 2: Load document content using LangChain's PDF loader
    try:
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()
    except PdfReadError as e:
        return None  # Return None if the PDF cannot be parsed

    # Step 3: Split document into overlapping text chunks
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(docs)

    # Step 4: Generate vector embeddings for each chunk using Google GenAI
    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

    # Step 5: Create a FAISS vectorstore from the embedded chunks
    vectorstore = FAISS.from_documents(chunks, embeddings)

    # Step 6: Return a retriever for semantic similarity search
    return vectorstore.as_retriever()


### `demo_manual_fetch(model_name: str, download_dir: str = "./downloads")`

**Purpose:**  
Fetches and downloads the English PDF user manual for a given Samsung model by orchestrating a series of steps from webpage scraping to file saving.

---

**Why is this needed?**

- **End-to-End Automation:**  
  This function acts as a single entry point to automate the retrieval of device manuals, combining web scraping, parsing, filtering, and downloading in one go.

- **Rapid Testing & Debugging:**  
  Originally designed as a demo or utility function to quickly validate that a manual can be found, filtered (English + PDF), and downloaded properly.

- **Ensures Accuracy:**  
  By previewing the list of all manual entries and only downloading English "user manual" PDFs, it guarantees relevance and language appropriateness.

---

**How does this work?**

1. **Construct Page URL:**
   - Forms the Samsung support page URL using the given `model_name`.

2. **Fetch Webpage & Extract JSON:**
   - Requests the webpage HTML and tries to extract the `"manuals"` array using `_extract_json_array`.
   - If the array doesn't exist or the page fails to load, the function returns early.

3. **Preview Available Manuals (Commented):**
   - Iterates through the manuals and prints (commented out) available languages and filenames for inspection.

4. **Select Target Manual:**
   - Uses `get_manual_pdf_url()` to find the first English PDF titled "user manual".
   - If not found, exits early.

5. **Download Selected Manual:**
   - Constructs the output path and downloads the manual using `download_manual()`.

6. **Return File Path:**
   - Returns the final local path of the downloaded PDF if everything succeeds.

This function is essential during development, testing, or bulk-fetching routines for acquiring device documentation cleanly and efficiently.


In [ ]:
def demo_manual_fetch(model_name: str, download_dir: str = "./downloads"):
    #print(f"→ Testing model: {model_name}\n")

    # 1) Raw fetch + JSON‐array extract
    page_url = f"https://www.samsung.com/in/support/model/{model_name}/"
    try:
        resp = requests.get(page_url, headers={"User-Agent":"Mozilla/5.0"}, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        #print(f"[Error] Failed to fetch page: {e}")
        return

    try:
        manuals = _extract_json_array(resp.text, "manuals")
        #print(f"Found {len(manuals)} total entries in the “manuals” array.")
    except RuntimeError:
        #print("No “manuals” array at all on the page.")
        return

    # Print out each entry’s languages & filename
    for idx, item in enumerate(manuals, start=1):
        langs = [lang.get("code") for lang in item.get("languageList",[])]
        fname = item.get("fileName")
        #print(f"  {idx}. {fname:30} langs={langs}")

    # 2) Pick the first English PDF “user manual”
    result = get_manual_pdf_url(model_name)
    if result is None:
        #print("\n→ No English PDF user-manual could be selected.")
        return
    href, fname = result
    #print(f"\nSelected English manual:\n  URL = {href}\n  filename = {fname}")

    # 3) Attempt download
    out_path = os.path.join(download_dir, fname)
    ok = download_manual(href, out_path)
    return out_path
    


### `load_manual(model_name: str, download_dir: str = "./downloads")`

**Purpose:**  
Builds a complete Retrieval-Augmented Generation (RAG) pipeline for a Samsung model by downloading its manual, extracting diagrams, and setting up an LLM-powered QA system.

---

**Why is this needed?**

- **One-Stop Setup for QA:**  
  Combines all the key components—manual fetching, diagram extraction, document indexing, and QA chain creation—into a single callable function.

- **Enables Question Answering:**  
  Equips the system to answer user queries specifically from the content of the manual, using a combination of semantic retrieval and LLM generation.

- **Foundation for RAG UI or Backend:**  
  Can be used as the backend logic for apps that allow users to interact with product manuals via natural language.

---

**How does this work?**

1. **Download the Manual:**
   - Calls `demo_manual_fetch()` to retrieve the English PDF manual.
   - Raises an error if no valid manual is found.

2. **Extract Diagrams:**
   - Uses `extract_diagrams()` to process visual content from the manual pages and save them as PNG images.

3. **Build Vector Retriever:**
   - Constructs a FAISS-based retriever by chunking the manual and embedding the text with Google GenAI embeddings.

4. **Initialize LLM:**
   - Creates a `GenAILLM` instance using the provided API key (`secret`).

5. **Create RAG Chain:**
   - Wraps the LLM and retriever in a `RetrievalQA` chain using the "stuff" method (simple concatenation of retrieved context).
   - Injects a custom prompt to guide response formatting and reasoning.

6. **Return Everything:**
   - Returns a tuple:
     - `chain`: the QA pipeline,
     - `pdf_path`: path to the downloaded manual,
     - `diagrams`: dictionary mapping page numbers to image file paths.

This function powers the core of the GenAI Samsung manual assistant—turning product documentation into an interactive, visual, and intelligent experience.


In [ ]:
def load_manual(model_name: str, download_dir: str = "./downloads"):
    """
    Loads a Samsung model's user manual, builds the retriever and LLM-based QA chain,
    and extracts associated diagrams.

    Args:
        model_name (str): The Samsung device model number (e.g., "SM-A166PLGBINS").
        download_dir (str): Directory to store the downloaded manual.

    Returns:
        Tuple: (QA chain, PDF path, diagrams dictionary)
            - chain: A RetrievalQA chain that can answer user queries.
            - pdf_path: Path to the downloaded user manual.
            - diagrams: Dictionary mapping page numbers to extracted diagram images.
    
    Raises:
        ValueError: If no English PDF manual is available for the given model.
    """

    # Step 1: Fetch and download the English PDF manual
    pdf_path = demo_manual_fetch(model_name, download_dir)
    if not pdf_path:
        raise ValueError(f"No English PDF manual available for model '{model_name}'.")

    # Step 2: Extract diagrams (images or page renderings) from the manual
    diagrams = extract_diagrams(pdf_path)

    # Step 3: Build a retriever for the manual using chunked embeddings
    retriever = build_retriever(pdf_path)

    # Step 4: Initialize the custom GenAI LLM with your secret API key
    llm = GenAILLM(api_key=secret)

    # Step 5: Create a Retrieval-Augmented Generation (RAG) chain
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Concatenates retrieved context before passing to LLM
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},  # Use custom prompt template
        return_source_documents=True  # Include source docs for traceability
    )

    # Step 6: Return the assembled QA chain, manual PDF path, and extracted diagrams
    return chain, pdf_path, diagrams


### `answer_query(question: str, chain, pdf_path: str, diagrams: Dict[str, List[str]])`

**Purpose:**  
Executes a natural language query against a product manual using an LLM-powered RAG pipeline, and returns both the structured answer and preview images of the referenced manual pages.

---

**Why is this needed?**

- **Interactive QA with Visuals:**  
  Enhances LLM responses by pairing text answers with relevant visual previews from the actual manual.

- **Structured & Clean Output:**  
  Uses the structured parser to extract a clean answer and ensures UI previews reflect only the most relevant content.

- **Context-Aware Retrieval:**  
  Provides accurate answers grounded in retrieved source documents, boosting both relevance and trustworthiness.

---

**How does this work?**

1. **Clear Old Previews:**
   - Empties the `PREVIEW_DIR` to prevent overlap with previous queries.

2. **Run QA Chain:**
   - Sends the user’s question to the LLM retrieval chain.
   - Parses the structured result using `output_parser`.

3. **Identify Source Pages:**
   - Collects and sorts page numbers from the retrieved source documents.
   - These pages are considered contextually relevant for the answer.

4. **Generate Page Previews:**
   - Opens the PDF and renders each relevant page as a high-resolution image.
   - Saves each image to `PREVIEW_DIR` and appends its path to `previews`.

5. **Append Extra Info:**
   - If pages were previewed, appends a note like:  
     `"✅ Preview pages 3–4."`

6. **Return Final Output:**
   - Returns a tuple:
     - `answer`: the parsed and enhanced textual response,
     - `previews`: a list of image paths for display.

This function bridges the gap between AI-generated answers and human-readable documentation by merging semantic understanding with visual cues from the manual.


In [ ]:
def answer_query(
    question: str,
    chain,
    pdf_path: str,
    diagrams: Dict[str, List[str]]
) -> Tuple[str, List[str]]:
    """
    Handles user question-answering by querying the RAG pipeline, parsing the response,
    and generating preview images for the relevant PDF pages.

    Args:
        question (str): User's natural language query.
        chain: The RetrievalQA chain used to generate answers.
        pdf_path (str): Path to the manual PDF file.
        diagrams (Dict[str, List[str]]): Dictionary mapping page numbers to diagram image paths.

    Returns:
        Tuple[str, List[str]]: 
            - A well-structured textual answer.
            - A list of preview image paths from the relevant pages.
    """

    # Step 1: Clear any previous preview images to avoid clutter
    for fname in os.listdir(PREVIEW_DIR):
        os.remove(os.path.join(PREVIEW_DIR, fname))

    # Step 2: Execute the RAG chain with the user's question
    result = chain.invoke({"query": question})

    # Step 3: Parse the model output using the structured output parser
    parsed = output_parser.parse(result.get("result") or result.get("output_text", ""))

    # Step 4: Extract page numbers from the source documents used in the response
    pages = sorted({
        int(doc.metadata.get("page", 0))
        for doc in result["source_documents"]
        if doc.metadata.get("page")
    })

    # Step 5: Generate image previews for each relevant page
    previews = []
    pdf = fitz.open(pdf_path)
    for page_num in pages:
        pix = pdf[page_num - 1].get_pixmap(matrix=fitz.Matrix(2, 2))  # High-res rendering
        preview_path = os.path.join(PREVIEW_DIR, f"page{page_num}_preview.png")
        pix.save(preview_path)
        previews.append(preview_path)
    pdf.close()

    # Step 6: Append preview page info to the answer if available
    extra = f"\n\n✅ Preview pages {pages[0]}–{pages[-1]}." if pages else ''
    answer = f"{parsed['answer']}{extra}"

    # Step 7: Return the final answer and the list of preview image paths
    return answer, previews


### `main()`

This function defines and launches the Gradio-based user interface for the Samsung Manual RAG QA system. It allows users to input a Samsung model number, load the corresponding manual, and ask natural language questions—returning AI-generated answers along with visual previews of the relevant manual pages. This interactive interface bridges user queries with document-driven GenAI responses in a clean and user-friendly way.

#### To test this application here are the sample model numbers:

1. SM-A166PLGBINS
2. QA55LS03FAULXL
3. NP754XGK-LS2IN
4. HMX-F90WP/MEA
5. NP-N102S-B05IN

In [ ]:
def main():
    """
    Launches the Gradio interface for the Samsung Manual RAG QA system.
    Allows users to load a Samsung manual by model number and ask natural language questions
    with AI-powered responses and visual page previews.
    """

    # Initialize a Gradio Blocks interface for composing multiple UI components
    demo = gr.Blocks()
    with demo:
        # Title / Header
        gr.Markdown("""## Samsung DocuMate
                    ### Your AI companion for Samsung docs.""")

        # Input: Samsung model number (restricted to 15 characters max)
        model_input = gr.Textbox(
            label="Model Number",
            max_lines=1,
            max_length=20,
            placeholder="Enter Model Number (max 20 chars)"
        )

        # Button to trigger manual loading
        load_btn = gr.Button('Load Manual')

        # Display the loading status (success or error)
        status = gr.Textbox(label='Status')

        # Non-editable file component to display the downloaded manual
        pdf_file = gr.File(label='Manual PDF', interactive=False)

        # Input field for natural language questions
        question_input = gr.Textbox(label='Your Question')

        # Button to trigger answering of the question
        ask_btn = gr.Button('Ask')

        # Output: Textbox to show the AI-generated answer
        response_box = gr.Textbox(label='Response')

        # Output: Gallery to show preview images of relevant PDF pages
        preview_gallery = gr.Gallery(label='Page Preview(s)', type='filepath')

        # State variables to preserve the chain, PDF path, and diagrams across interactions
        chain_state, path_state, diag_state = gr.State(), gr.State(), gr.State()

        # Logic to handle manual loading when 'Load Manual' button is clicked
        def on_load(model: str):
            try:
                chain, path, diagrams = load_manual(model)  # Build QA system for the model
                return f'Loaded: {model}', chain, path, diagrams, path
            except ValueError as e:
                return str(e), None, None, None, None

        # Link the 'Load Manual' button to the on_load function
        load_btn.click(
            fn=on_load,
            inputs=[model_input],
            outputs=[status, chain_state, path_state, diag_state, pdf_file]
        )

        # Link the 'Ask' button to the answer_query function
        ask_btn.click(
            fn=answer_query,
            inputs=[question_input, chain_state, path_state, diag_state],
            outputs=[response_box, preview_gallery]
        )

    # Launch the Gradio app
    demo.launch()

# Entry point for the script
if __name__ == '__main__':
    main()
